## 1. Load state model

In [2]:
from ofact.planning_services.model_generation.persistence import deserialize_state_model
from ofact.twin.state_model.model import StateModel

Current working directory: C:\Users\afreiter\PycharmProjects\ofact-intern\projects\tutorial\03_analytics
Resolved ROOT_PATH: C:\Users\afreiter\PycharmProjects\ofact-intern\ofact
Resolved state model file path: C:\Users\afreiter\PycharmProjects\ofact-intern\ofact\projects\iot_factory_simulation\models\twin\mini.xlsx
No module named 'pyarrow'


In [3]:
state_model_file_path = "../../tutorial/updated_state_model.pkl"
state_model: StateModel = deserialize_state_model(state_model_file_path, persistence_format="pkl",
                                                  dynamics=True)


Platform name: Windows


In [4]:
a = state_model.get_resource_capacity_utilization(["Main Part AGV 1"])
b = state_model.get_order_lead_time()
c = state_model.get_delivery_reliability()
print(a)
print(b)
print(c)

[1.9251131003946114e-14]
3:36:37.500000
0.5


## 2. Show KPI's

In [4]:
from datetime import datetime
from ofact.planning_services.scenario_analytics.scenario_handling.single import SingleScenarioHandler

scenario_name = "current_state"
single_scenario_handler = SingleScenarioHandler(state_model, scenario_name=scenario_name)

start_time = datetime(2024, 10, 22, 8)
end_time = datetime(2024, 10, 25, 8)
start_time_timestamp = start_time.timestamp()
end_time_timestamp = end_time.timestamp()
available_ids_dict = single_scenario_handler.get_filter_options(start_time=start_time_timestamp, end_time=end_time_timestamp)
order_ids_list, product_ids_list, process_ids_list, resource_ids_list = (
    list(available_ids_dict["order"].index), list(available_ids_dict["product"].index),
    list(available_ids_dict["process"].index), list(available_ids_dict["resource"].index))

# order lead time
lead_time_df = single_scenario_handler.get_lead_time(start_time=start_time, end_time=end_time,
                                                     order_ids_list=order_ids_list, product_ids_list=product_ids_list,
                                                     process_ids_list=process_ids_list, resource_ids_list=resource_ids_list,
                                                     event_type="ACTUAL", view="ORDER")

print("Available column names:", lead_time_df.columns.values)
print("\n")
total_lead_time_mean = round(lead_time_df["avg_lead_time"].mean())
print(f"The average total lead time (without waiting time) of the orders is {total_lead_time_mean} minutes.")
total_lead_time_wt_mean = round(lead_time_df["total_lead_time_wt"].mean(), 2)
print(f"The average total lead time (with waiting time) for orders is {total_lead_time_wt_mean} minutes, \n",
      f"including a waiting time of {total_lead_time_wt_mean - total_lead_time_mean}, \n",
      f"which is {round(total_lead_time_wt_mean / total_lead_time_mean, 1)} times the processing time")


AnalyticsDataBase instantiation finished
Available column names: ['ID' 'View' 'Start Time [s]' 'End Time [s]' 'avg_lead_time'
 'avg_waiting_time' 'avg_lead_time_wt' 'total_lead_time'
 'total_waiting_time' 'total_lead_time_wt' 'min_lead_time'
 'min_waiting_time' 'min_lead_time_wt' 'max_lead_time' 'max_waiting_time'
 'max_lead_time_wt' 'var_lead_time' 'var_waiting_time' 'var_lead_time_wt']


The average total lead time (without waiting time) of the orders is 65 minutes.
The average total lead time (with waiting time) for orders is 208.26 minutes, 
 including a waiting time of 143.26, 
 which is 3.2 times the processing time


## 3. Show KPI's in Excel

Export the KPI's in Excel files.


In [5]:
start_time = datetime(2024, 10, 22, 8)
end_time = datetime(2024, 10, 22, 12)
orders_df, products_df, processes_df, resources_df, resource_utilization_df = (
                single_scenario_handler.get_all_kpis(start_time=start_time, end_time=end_time))

orders_df.to_excel("orders.xlsx", index=False)
products_df.to_excel("products.xlsx", index=False)
processes_df.to_excel("processes.xlsx", index=False)
resources_df.to_excel("resources.xlsx", index=False)

[API] filter requested
